In [3]:
%pylab
import imageio
import glob
import pandas
from tqdm.notebook import tqdm
import scipy.ndimage as ndi
from skimage import morphology
from mayavi import mlab
from os.path import basename

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [1]:
rdir = "/media/maxc/IcyBox/Cooper2021_Data/"
sname = "PIN43"
sdir = f"{rdir}{sname}/"

In [ ]:
recondir = f"{sdir}/4D/"
ilen = 3
refnum = "000"
subdir = f"{sdir}Subtractions/"

In [ ]:
totimgs = len(glob.glob(f"{recondir}*.tif"))
reff = f"{recondir}{sname}_{refnum}.tif"
refimg = imageio.volread(reff)
print(f"Loaded reference image: {reff}")

for i in arange(totimgs): # iterate over non-reference time slices
    wnum = f"{i:0>ilen}" # working time slice name
    wf = f"{recondir}{sname}_{wnum}.tif"
    if  wf == reff:
        continue

    wimg = imageio.volread(wf) # working image
    print(f"Loaded working image: {wf}")
    # Subtract, if overflow uint16 -> pixel = 0 (ImageJ subtract behaviour)
    res = refimg - wimg
    res[refimg < wimg] = 0
    ofile = f"{subdir}{sname}_{refnum}-{wnum}.tif"
    imageio.volwrite(ofile, res)
    print("Wrote image: " + ofile)

In [2]:
bdir = f"{sdir}HDSkeletons/"
cnum = 8
cname = f"{sname}_{refnum}-{cnum:0>ilen}.tif"
print(cname)

PIN43_01-08.tif


In [6]:
i = imageio.volread(f"{subdir}{cname}")
#i = i[fs:ls+1] # crop
i = ndi.median_filter(i, (3,3,3)) # replace voxels with median of 3x3x3 window
bi = i>=15000 # binary image from threshold
bi = ndi.binary_fill_holes(bi) #fill holes in bi
skel = morphology.skeletonize_3d(bi) # skeletonize by morphological thinning

### Filters
## Filter by counting number of skeleton voxels in window.
# If less than number permitting to be part of wormhole, gives false.
ws = 9
wind = ones((ws,ws,ws))
fval = 5
filtered_skel = ndi.convolve(skel.astype(bool), wind, output=uint8, mode='constant', cval=0)>=fval
filtered_skel = (filtered_skel&skel).astype(bool) # keep voxels where skeleton voxel meets filter params

## Pass #2 with window to filter isolated voxels that break conversion to graph
ws2 = 3
wind = ones((ws2,ws2,ws2))
filtered_skel = ndi.convolve(filtered_skel, wind, output=uint8, mode='constant', cval=0)>=1
filtered_skel = (filtered_skel&skel).astype(bool)

## Pass #3, isolated fragments or noise are OK if they are near/below the w.h. tip in z-dir.
# However, if they are above they can throw off Z_max estimation

frag = True
while(frag):
    zmax = where(any(filtered_skel, axis=(1,2)))[0].max() # Maximum z index where skel img contains a voxel
    frag = ~any(filtered_skel[zmax-fval]) # Check if slice fval below zmax contains skeleton
    if frag:
        filtered_skel[zmax-fval:] = 0 # Remove fragment
    else: break

In [7]:
fig = mlab.figure(size=(600,1080))
vol = mlab.pipeline.volume(mlab.pipeline.scalar_field((filtered_skel&skel).T))
vol.volume_mapper_type = 'FixedPointVolumeRayCastMapper'
vol.volume_property.shade = False

In [8]:
imageio.volwrite(f"{bdir}{cname}", \
                 (filtered_skel&skel).astype(bool).astype(uint8))

In [ ]:
recondir = bdir
flist = sorted(glob.glob(f"{recondir}*.tif"))
skimgs = [imageio.volread(i) for i in flist]

In [ ]:
zmax = zeros(len(flist)) # Z_max for time slices

skelinit = 0
sf = False

while not sf:
    if sum(skimgs[skelinit]) < 1:
        skelinit +=1
    else: break

for k, fname in enumerate(tqdm(flist[skelinit+1:])):
    i = k+skelinit
    zmax[i+1] = where(any(skimgs[i+1], axis=(1,2)))[0].max() # Z_max of current working skeleton

In [ ]:
flist = [basename(i) for i in flist]
df = pandas.DataFrame({"Scan":flist, "Z_max":zmax})
plot(zmax)
show()
df.to_csv("PIN43_Zmax.csv")